In [4]:
import cv2  #Görüntü işleme için OpenCV
from ultralytics import YOLO  # Cisimleri algılama için Ultralytics'den YOLO

model = YOLO('yolov8n.pt')  # YOLO'nun v8-Nano Modeli

cap = cv2.VideoCapture('test.mp4')  # Videoyu aç

fgbg = cv2.createBackgroundSubtractorMOG2()  # Arka plan çıkarma işlemi (Daha rahat hareket algılamak için) 

#Tüm kareler üzerinde işlem yapabilmek için frameleri döngüye sokuyoruz
while True:
    ret, frame = cap.read() #Her seferinde bir sonraki kareyi almak için (Ret -> return (True ya da False), frame -> Kare)
    if not ret: #Başka kare yoksa yani video bittiyse döngüyü bitir
        break

    
    fgmask = fgbg.apply(frame) #Arka plan çıkarma ve kontür bulma işlemi için maskeleme uygulanır. (Beyaz hareketli, siyah hareketsiz)
    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) #Hareket maskesindeki beyaz bölgeleri konturlara çevirir.

    results = model(frame) #Yolo modelinin frame üzerinde tahmin yapması

#Tahminlerin kare içine alınarak belirtilmesi
    #Her obje için hareket kontrolü
    for result in results:
        for box in result.boxes.xyxy:
            x1, y1, x2, y2 = map(int, box)
            #Obje ile hareketli kontur çakışıyor mu?
            motion_detected = False
            for contour in contours:
                if cv2.contourArea(contour) < 500:
                    continue
                (mx, my, mw, mh) = cv2.boundingRect(contour)
                if (x1 < mx + mw and x2 > mx and y1 < my + mh and y2 > my):
                    motion_detected = True
                    break
            #Çizim ve etiketleme
            if motion_detected:
                color = (0, 0, 255)  # Kırmızı (hareketli)
                label = 'MOTION'
            else:
                color = (0, 255, 0)  # Yeşil (sabit)
                label = 'STATIC'

            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    cv2.imshow('YOLOv8 + Hareket Tahmini', frame) #Tahminlerle birlikte gerçek görüntüyü göster

    cv2.imshow('FGMask', fgmask) #Maskelenmiş kontürleri çizilmiş şekilde göster (beyaz = hareket, siyah = arka plan)

    #Arka plan temizlenmiş görüntü
    masked_frame = cv2.bitwise_and(frame, frame, mask=fgmask)
    cv2.imshow('A', masked_frame)

    #Q tusuna basarak döngüden çıkmak - görüntüyü kapatmak
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 352x640 8 cars, 46.0ms
Speed: 5.2ms preprocess, 46.0ms inference, 5.4ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 8 cars, 26.7ms
Speed: 6.4ms preprocess, 26.7ms inference, 7.4ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 8 cars, 14.2ms
Speed: 5.3ms preprocess, 14.2ms inference, 3.6ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 8 cars, 16.4ms
Speed: 2.5ms preprocess, 16.4ms inference, 3.6ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 8 cars, 14.4ms
Speed: 2.9ms preprocess, 14.4ms inference, 4.1ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 8 cars, 15.5ms
Speed: 3.0ms preprocess, 15.5ms inference, 6.1ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 8 cars, 12.9ms
Speed: 2.4ms preprocess, 12.9ms inference, 2.6ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 8 cars, 12.9ms
Speed: 2.5ms preprocess, 12.9ms inference, 4.7ms postprocess per image at shape (1, 3, 352, 640)

0: 352x